# Using the Spotify API to Search for and Get Tracks from a Playlist

This notebook goes through the basic process of using [Spotipy](https://spotipy.readthedocs.io/en/2.11.1/) to access the Spotify API and (as the title says) search for a playlist and get all its tracks (in the case of this notebook, that means printing them out -- a later notebook will go through actually getting audio features and other info about these tracks).

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Below you can see how to use the [Client Credentials Flow](https://spotipy.readthedocs.io/en/2.11.1/#client-credentials-flow) to access the API. I followed the details in this [post](https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b) to set up my credentials.

In [2]:
cid = # your client id
secret = # your client secret

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Let's now go ahead and search for a playlist of [Pitchfork's Top 200 Tracks of the 1990s](https://pitchfork.com/features/lists-and-guides/7854-the-top-200-tracks-of-the-1990s-200-151/) (this will return 10 results by default but we'll only care about the very first of these):

In [4]:
playlist_search = sp.search(q='pitchfork songs of the 1990s', type='playlist')

OK, now let's get the [URI](https://community.spotify.com/t5/Spotify-Answers/What-s-a-Spotify-URI/ta-p/919201) for the and title of first search result, just to sanity check:

In [5]:
pfork90uri = playlist_search['playlists']['items'][0]['uri']
pfork90uri

'spotify:playlist:2QISjibz6qfirtz7okUSHV'

In [6]:
pfork90name = playlist_search['playlists']['items'][0]['name']
pfork90name

"Pitchfork's Top 200 Songs of the 1990s"

Now let's get the tracks from this playlist:

In [7]:
pfork90s = sp.playlist_tracks(pfork90uri)
len(pfork90s['items'])

100

OK, we were limited to 100 tracks, but clearly there are 200 tracks, it says so in the name of the playlist, right? Fortunately we can use the `next` field from the results of our search to get the rest of the tracks as demonstrated in this [example](https://github.com/plamere/spotipy/blob/master/examples/user_playlists_contents.py) (yes, this is not all original code created by me and adapted for my purposes).

In [8]:
pfork90s['next']

'https://api.spotify.com/v1/playlists/2QISjibz6qfirtz7okUSHV/tracks?offset=100&limit=100&additional_types=track'

In [9]:
def print_playlist_tracks(playlist, run):
    for i, track in enumerate(playlist.get('items', [])):
        print(str((100*run) + (i+1)), track['track']['name'], '-', track['track']['artists'][0]['name'])

In [10]:
run = 0
print_playlist_tracks(pfork90s, run)
while pfork90s['next']:
    run += 1
    pfork90s = sp.next(pfork90s)
    print_playlist_tracks(pfork90s, run)

1 Gold Soundz - Pavement
2 Common People - Pulp
3 Nuthin' but a "G" Thang - Dr. Dre
4 Paranoid Android - Radiohead
5 Protect Ya Neck - Wu-Tang Clan
6 Only Shallow - My Bloody Valentine
7 Holland, 1945 - Neutral Milk Hotel
8 Are You That Somebody - Aaliyah
9 Loser - Beck
10 Say It Ain't So - Weezer
11 Hyperballad - Björk
12 Windowlicker - Aphex Twin
13 Smells Like Teen Spirit - Nirvana
14 Juicy - 2005 Remaster - The Notorious B.I.G.
15 Enjoy the Silence - Single Mix - Depeche Mode
16 SpottieOttieDopaliscious - OutKast
17 The State I’m In - Belle & Sebastian
18 Da Funk - Daft Punk
19 Fade Into You - Mazzy Star
20 Midnight In A Perfect World - DJ Shadow
21 1979 - Remastered 2012 - The Smashing Pumpkins
22 Cannonball - The Breeders
23 Ladies and Gentlemen We Are Floating in Space - Spiritualized
24 Car - Built To Spill
25 Shook Ones, Pt. II - Mobb Deep
26 Girls and Boys - 2012 Remaster - Blur
27 Needle In The Hay - Elliott Smith
28 It Ain't Hard to Tell - Nas
29 Bitter Sweet Symphony - The

OK it seems some of these rankings are off by 1 at the end, since there's 201 tracks here -- oh wait it even says it in the description!

In [13]:
playlist_search['playlists']['items'][0]['description']

'Note: There appears to be one too many songs because the list included Part 1 and Part 2 of &quot;Trip II The Moon&quot; by Acen.'